# Baseline RAG テスト

こちらを参考にBaseline RAGを構築TEST：

https://note.com/noa813/n/n6065209e7bf0

### ライブラリーのインストール
!pip install langchain openai chromadb tiktoken

### OpenAIのAPIキーを環境変数に格納

In [2]:
import openai
import os
import getpass

# !初回のみ! openaiのAPIキーを環境変数として設定

# OpenAI APIのアカウントを作成、
# https://platform.openai.com/account/api-keys
# からAPIキーを取得して環境変数に設定

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

# openai.api_key = os.getenv("OPENAI_API_KEY")

### LLMが参照するテキストを用意

In [ ]:
# file_path = './data/manufacturing_company_dummy_v2.txt'
file_path = './data/GRPHRG株式会社_社員スキル.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text


### テキストを分割

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="。\n\n",
    chunk_size=200, 
    chunk_overlap=0
    )

docs = text_splitter.split_text(text)

print(f"{len(docs)} chunks")
docs

### OpenAIのEmbeddingのインスタンス化

* チャンクのベクトル化を行うためのインスタンス作成
* 追加インストールが必要：pip install -U langchain-community

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

### VectorStore作成
これがベクトル化されたチャンクを保存するためのベクトルDB

In [6]:
from langchain.vectorstores import Chroma
# 「chromaDB」:ベクトル型ストアで代表的なデータベース
db = Chroma.from_texts(docs, embeddings)

# #取得するチャンク数を変更(通常は4)
n_chunks = 10
retriever = db.as_retriever(search_kwargs={"k": n_chunks})

### 質問（Query）を用意

質問（query）がベクトル化され、
queryと類似度が近いベクトルを持つ
テキストをVectorStoreから検索します。
そして、
queryに関連する文書が
context_docsへ格納されます

In [ ]:
# query = "人的資本の強化のために取り組んでいることは何でしょうか？"
# query = "田中太郎と一緒にプロジェクトに従事したことがある社員を全員挙げてください"
# query = "この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。"
# query = """
#             この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}を一つ以上持つ社員を10名選定せよ。
#             {#スキル}
#             1. プロジェクトマネジメント
#             2. データ解析
#             3. システム開発
#             4. プロセス改善
#             5. 現場の技術知識
#             """   
# query = """この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}を持つ社員をちょうど10名選定し、プロジェクトにおける役割を明確にせよ。
#             {#スキル}
#             1. プロジェクトマネジメント
#             2. データ解析
#             3. システム開発
#             4. プロセス改善
#             5. 現場の技術知識"""
# query = "この文章の内容から、この企業の社員のスキルはどういうものが多いのかを整理してください。"
# query = "マーケティングの知識が最も豊富だと考えられる社員は誰ですか？"
# query = "佐々木健太さんはどういった社員ですか？"
# query = "藤井健太さんと近いスキルセットを持つ社員と、そのスキルセットについてそれぞれ教えてください。"

# query = "データ分析のスキルを持つ社員を全員リストアップしてください。"
# query = "プロジェクトマネジメントの経験を持つ社員を全員リストアップしてください。"
# query = "藤田玲奈のTOEICスコアは何点ですか？"

# query = "全社員についての情報の中で、プログラミング言語は何種類リストアップされてますか？"

query = "プロジェクトマネジメント資格（PMP）を保有している社員は何名いますか？"




context_docs = retriever.get_relevant_documents(query)
context_docs

### *** LLMに質問を投げ、RAGを使って回答

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

result = qa_chain.run(query)
print(result)

### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」

------

##### ダミーデータ(製造業企業の社員情報 v2) で検証
**Query #1:「この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。」**

**LLMの回答**

* turbo-3.5

「### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」


Query #2:「この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}を一つ以上持つ社員を10名選定せよ。
            {#スキル}
            1. プロジェクトマネジメント
            2. データ解析
            3. システム開発
            4. プロセス改善
            5. 現場の技術知識

**LLMの回答**

* 4o

以下の社員が生産ラインのDXプロジェクトを成功させるために必要なスキルを持っています。

1. 田中 太郎
   - スキル: プロジェクトマネジメント、プロセス改善、現場の技術知識

2. 三浦 夏美
   - スキル: プロセス改善、現場の技術知識

3. 加藤 直樹
   - スキル: プロセス改善、現場の技術知識

4. 平野 大輝
   - スキル: データ解析

この4名が該当スキルを持っています。<span style="background-color: red;">他の社員についての情報がないため、10名を選定することはできません。</span>



-----


#### skillダミーデータで検証


Query #1:「この文章の内容から、この企業の社員のスキルはどういうものが多いのかを整理してください」

**LLMの回答**

* 4o

<取得された関連データ>
[Document(metadata={}, page_content='山本幸子は、人事部で採用と研修を担当しており、社会保険労務士の資格を持つ。組織開発や社員教育プログラムの企画・運営に携わり、社員のスキルアップと職場環境の改善に貢献している。労働法にも精通し、コンプライアンス遵守の推進役でもある'),
 Document(metadata={}, page_content='山本幸子は、人事部で採用と研修を担当しており、社会保険労務士の資格を持つ。組織開発や社員教育プログラムの企画・運営に携わり、社員のスキルアップと職場環境の改善に貢献している。労働法にも精通し、コンプライアンス遵守の推進役でもある'),
 Document(metadata={}, page_content='山本幸子は、人事部で採用と研修を担当しており、社会保険労務士の資格を持つ。組織開発や社員教育プログラムの企画・運営に携わり、社員のスキルアップと職場環境の改善に貢献している。労働法にも精通し、コンプライアンス遵守の推進役でもある'),
 Document(metadata={}, page_content='坂本美咲は、コンテンツマーケティングのスペシャリストであり、ブログやホワイトペーパーの制作を手がけている。ウェブ解析士の資格を持ち、SEOやコンバージョン最適化により集客力を高めている。コピーライティングのスキルも高い'),
 Document(metadata={}, page_content='坂本美咲は、コンテンツマーケティングのスペシャリストであり、ブログやホワイトペーパーの制作を手がけている。ウェブ解析士の資格を持ち、SEOやコンバージョン最適化により集客力を高めている。コピーライティングのスキルも高い'),
 Document(metadata={}, page_content='坂本美咲は、コンテンツマーケティングのスペシャリストであり、ブログやホワイトペーパーの制作を手がけている。ウェブ解析士の資格を持ち、SEOやコンバージョン最適化により集客力を高めている。コピーライティングのスキルも高い'),
 Document(metadata={}, page_content='中村亮太は、営業部のエースとして国内外の大手クライアントを担当している。ビジネス英語検定準1級を取得し、海外取引にも積極的だ。高度な交渉力とプレゼンテーションスキルで新規契約を次々と獲得している。CRMシステムの活用にも長けている'),
 Document(metadata={}, page_content='中村亮太は、営業部のエースとして国内外の大手クライアントを担当している。ビジネス英語検定準1級を取得し、海外取引にも積極的だ。高度な交渉力とプレゼンテーションスキルで新規契約を次々と獲得している。CRMシステムの活用にも長けている'),
 Document(metadata={}, page_content='中村亮太は、営業部のエースとして国内外の大手クライアントを担当している。ビジネス英語検定準1級を取得し、海外取引にも積極的だ。高度な交渉力とプレゼンテーションスキルで新規契約を次々と獲得している。CRMシステムの活用にも長けている'),
 Document(metadata={}, page_content='青木隆は、アナリティクス部でデータ分析とビジネスインサイトの提供を行っている。ビッグデータ解析の資格を持ち、機械学習アルゴリズムを用いた予測モデルの構築に成功している。データビジュアライゼーションのスキルも高い')]

 <回答>

 「1. **人事・組織開発**:
   - 採用と研修の担当
   - 社会保険労務士の資格
   - 組織開発や社員教育プログラムの企画・運営
   - 労働法の知識とコンプライアンス遵守の推進

2. **コンテンツマーケティング**:
   - ブログやホワイトペーパーの制作
   - ウェブ解析士の資格
   - SEOやコンバージョン最適化
   - コピーライティングのスキル

3. **営業**:
   - 国内外の大手クライアントの担当
   - ビジネス英語検定準1級
   - 高度な交渉力とプレゼンテーションスキル
   - CRMシステムの活用

4. **データ分析**:
   - データ分析とビジネスインサイトの提供
   - ビッグデータ解析の資格
   - 機械学習アルゴリズムを用いた予測モデルの構築
   - データビジュアライゼーションのスキル

このように、社員はそれぞれの専門分野で高度なスキルと資格を持ち、企業の多様なニーズに対応しています。」


**-><span style="background-color: red;">数人の社員のスキルを抜粋して整理しただけ。いまいち。</span>**




Query #2:「この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}のどれかを持つ社員をちょうど10名選定し、プロジェクトにおける役割を明確にせよ。
            {#スキル}
            1. プロジェクトマネジメント
            2. データ解析
            3. システム開発
            4. プロセス改善
            5. 現場の技術知識
         」

**LLMの回答**

* 4o

「<span style="background-color: red;">「申し訳ありませんが、提供された情報からは、GRPHRG株式会社の社員の中でデータ解析や現場の技術知識を持つ社員についての情報が不足しているため、プロジェクトに適した10名を選定することができません。プロジェクトマネジメントやシステム開発に関しては、佐藤一郎や加藤正志が適任かもしれませんが、他のスキルに関しては情報が不足しています。」</span>